In [1]:

import os
import pandas as pd
import re
from lxml import etree
import datetime
from datetime import datetime
import time

ns = {'tei': 'http://www.tei-c.org/ns/1.0', "xml": "http://www.w3.org/XML/1998/namespace"}




def readlist(pathtofile, filename):
      tree = etree.parse(pathtofile+"/"+filename)
      root = tree.getroot()
      return root

def readfiles(normrolelist, personlistxml, pathtofiles, type):
    Frame = pd.DataFrame()
    texts =[]
    norms = []
    persons = []
    sex = []
    relatedPersons =[]
    relatedSex = []
    regests = []
    regest_dates = []
    fromdate=[]
    todate=[]
    years = []
    listoferrors = []
    for filename in os.listdir(pathtofiles):
        print("--------------------------")
        print(filename)
        regesttree = etree.parse(pathtofiles+"/"+filename)
        regestroot = regesttree.getroot()
        actualregest = filename
        creationdate = regestroot.find(".//tei:creation/tei:date", ns)
        origindate = regestroot.find(".//tei:history/tei:origin", ns)
        if creationdate is not None:
           date = createdate(creationdate)
        elif origindate is not None:
           date = createdate(origindate)
        else:
           date = " "
        Nodes = regestroot.findall(".//tei:roleName[@type='"+type+"']", ns)
        for rs in Nodes:
            try:
                print("---------")
                print("-Node: %s" %rs.tag)
                print("--Attribute: %s" %rs.attrib)
                text = rs.xpath("normalize-space(.)")
                print("--Text: %s" %text)
                normed = normrole(normrolelist, text)

                if 'corresp' in rs.attrib:
                    relatedPerson = rs.attrib['corresp'].replace("#", "")
                elif 'ref' in rs.attrib:
                    relatedPerson = "ref"
                else:
                    relatedPerson = "None"
            
                personelement = rs.xpath("ancestor::*[@type='person'][1]")
                print("-ParentNode: %s" %personelement[0].tag)
                if personelement is not None:
                    print("--Attribute: %s" %personelement[0].attrib)
                    personref = personelement[0].attrib['ref'].replace("#","")
                else:
                    personref = " "
            except:
                errorarray =[filename, rs.tag, rs.attrib, rs.xpath("normalize-space(.//text())")]
                listoferrors.append(errorarray)
                writeerrorfile(listoferrors)




            texts.append(normed[0])
            norms.append(normed[1])
            persons.append(personref)
            sex.append(getsex(personlistxml, personref))
            relatedPersons.append(relatedPerson)
            if "pe__" in relatedPerson:
                relatedSex.append(getsex(personlistxml, relatedPerson))
            else:
                relatedSex.append(" ")
            regests.append(actualregest)
            if len(date) == 2:
                persdate = date
                regest_dates.append(persdate[0])
                fromdate.append("")
                todate.append("")
                years.append(persdate[1])
            elif len(date) == 3:
                fromtodate = date
                regest_dates.append("")
                fromdate.append(fromtodate[0])
                todate.append(fromtodate[1])
                years.append(fromtodate[2])
    
    
    
    Frame["Person"] = persons
    Frame['sex'] = sex
    Frame[type] = texts
    Frame["normed"] = norms
    Frame["Related Person"] = relatedPersons
    Frame['rel_sex'] = relatedSex
    Frame["xml"] = regests
    Frame["Year"] = years
    Frame["Date"] = regest_dates
    Frame["from"] = fromdate
    Frame["to"] =todate


    Frame.to_csv("../data/"+type+"Relations.csv", sep="\t", encoding="utf-8")

    





def writeerrorfile(listoferrors):
    with open("../data/output/relations_ERROR.txt", "w") as file:
        if len(listoferrors) > 0:
            file.write(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
            file.write('\n')
            for error in listoferrors:
                file.write("Error in: %s \n" %error[0] )
                file.write("-Node %s \n" %error[1])
                file.write("-attributes: ")
                for attribute in error[2]:
                    file.write("%s; " %attribute)
                file.write("\n")
                file.write('-Nodetext: "%s"\n' %error[3])
                file.write("-------------\n")
        else:
            file.write(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
            file.write("\nKeine ERRORS gefunden")


def createdate(node):
    if 'when' in node.attrib:
        when = node.attrib['when'].replace('-','')
        date = datetime.strptime(when, '%Y%m%d').date()
        year = str(date).split("-")[0]
        return [date,year]

    elif 'notAfter' in node.attrib:
        notafter = node.attrib['notAfter']
        if "-" in notafter:
            date = datetime.strptime(notafter, '%Y-%m-%d').date()
        else:
            date = datetime.strptime(notafter, '%Y').date()
        year = str(date).split("-")[0]
        return [date, year]
    elif 'from' in node.attrib:
        fr =node.attrib['from'].replace('-','')
        to = node.attrib['to'].replace('-','')
        fromdate = datetime.strptime(fr, '%Y%m%d').date()
        todate = datetime.strptime(to, '%Y%m%d').date()
        year = str(fromdate).split("-")[0]
        if fromdate == todate:
            return[fromdate,year]
        else:
            return [fromdate, todate,year]
    


def getrolenormlist():
    role_mapping = pd.read_csv("https://docs.google.com/spreadsheets/d/1_ygvqRnDTg6Rx2wp-RNPvZ3cKQFxfDPo3d7SBDiLO5c/export?gid=0&format=csv", dtype=str)\
    .iloc[:,1:3]
    rolesnormlist = role_mapping.set_index("role").to_dict()

    return rolesnormlist['role_norm']


def normrole(rolenormlist, role):
    if role.lower() in rolenormlist:
        return [role, rolenormlist[role.lower()]]
    elif role in rolenormlist:
        return [role, rolenormlist[role]]
    else:
        return [role, " "]


def getsex(personlistxml, id):
    return personlistxml.xpath(".//tei:person[@xml:id='"+id+"']", namespaces=ns)[0].attrib['sex']


normrolelist = getrolenormlist()
personlistxml = readlist("../../indices/lists", "personList.xml")


readfiles(normrolelist, personlistxml, "../../sources/Satzbuch_CD/1448/done", "off") #Pfad (Durchsuchung), @type of relation
print("done")

--------------------------
00054-eintrag_vom_1448-06-10.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien-st_agnes_auf_der_himmelpforte-altar_dreikoenige'}
--Text: caplan
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__simon_pirkwicz_SB_CD_00054'}
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien'}
--Text: rats
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__andreas_hiltprant_QGW_II_II_2576'}
--------------------------
00055-eintrag_vom_1448-06-21.xml
--------------------------
00056-eintrag_vom_1448-08-09.xml
--------------------------
00156-eintrag_vom_1448-12-13.xml
--------------------------
00226-eintrag_vom_1448-02-23.xml
--------------------------
00227-eintrag_vom_1448-03-06.xml
--------------------------
00228-eintrag_vom_1448-04-22.xml
--------------------------
00229-eintrag_vom_1